***Busqueda de N1 min y max***
$$\int_\Omega \nabla u \cdot \nabla v~dx = \int_\Omega fv~dx$$
$$B_{max} = \frac{1}{2}\frac{\int V_L(t)dt}{NA} + \frac{NI_{DC}}{RA} < B_{sat}$$
$$\frac{1}{2}\frac{V_dDT_s}{NA} + \frac{NI_{DC}}{RA} < B_{sat}$$

Con

$$R = \frac{l_e}{\mu_0 \mu_e A}$$

Desmos: https://www.desmos.com/calculator/x80nw1l7ab

Cosas de TME:
$$N_{min} = \frac{1}{2}\frac{V_dDT_s}{B_{sat}A_{min}}$$

In [46]:
from scipy.optimize import fsolve
import numpy as np

Bsat = 320E-3

#Datos
Vd = 300
#Ponemos un D alto para asegurarnos que ni asi va a saturar
D = 0.9
Fs = 100E3
Ts = 1/Fs
#Corriente DC maxima para el MOS
IDC = 3
#Datos del nucleo
Al = 5200E-9
Al_min = Al*(1-0.2)
Al_max = Al*(1+0.3)
A_min = 229E-6
le = 97E-3
u0 = 4*np.pi*1E-7
ue = 1690
Ae = 234E-6
R = le/(u0*ue*A_min)

#Ahora encontramos los parametros limite y deseados
N_min = 0.5*Vd*D*Ts/(Bsat*A_min)
print("N_min = ", N_min, " vueltas. Redondeamos a ", np.ceil(N_min))

#Este es el N_min para que el nucleo no sature por su propia inductancia, ahora tenemos que ver un Idc maximo admitible
#Resolvemos la ecuacion cuadratica: 0.5*Vd*D*Ts/(A_min) + (N**2)*IDC/(R*A_min) - Bsat*N = 0
coeffs = [IDC/(R*A_min), - Bsat, 0.5*Vd*D*Ts/(A_min)]
print("Raices:", np.roots(coeffs), ",\t Dan imaginarias, asi que la IDC es muy grande, probamos con algo mas bajo")
#Vemos que nos da raices complejas, asi que la IDC (que es el unico parametro que podemos mover, es muy alta)
#Este valor da una par de raices entendibles, con buen margen para error
IDC = 180E-3
coeffs = [IDC/(R*A_min), - Bsat, 0.5*Vd*D*Ts/(A_min)]
print("Raices:", np.roots(coeffs), ",\t Con IDC = 180mA, tomamos N1=30")
#Con esto, tendriamos que la corriente media en el primario es 180mA, lo que se traduce 180mA*N1/N2 = 2.25A en el secundario de 24V, no esta nada mal
#Teniendo en cuenta estos valores para N1, tomamos:
N1 = 30

#Resto de parametros del transformador
Vo2 = 24
Vo3 = 16
L1 = Al*N1**2
N2 = N1*Vo2/Vd
L2 = Al*N2**2
N3 = N1*Vo3/Vd
L3 = Al*N3**2
print("")
print("N1 = ", N1, " vueltas")
print("L1 = ", L1*1E3, "mHy")
print("N2 = ", N2, " vueltas")
print("L2 = ", L2*1E3, "mHy")
print("N3 = ", N3, " vueltas")
print("L3 = ", L3*1E3, "mHy")
print("")

print("Esta combinacion da valores de inductancias altas, pero peor es que da vueltas decimales, tenemos dos opciones:")
print("\tOpcion 1:")
IDC = 70E-3
coeffs = [IDC/(R*A_min), - Bsat, 0.5*Vd*D*Ts/(A_min)]
print("\tRaices:", np.roots(coeffs), ",\t Con IDC = 70mA")
N1 = 150

#Resto de parametros del transformador
Vo2 = 24
Vo3 = 16
L1 = Al*N1**2
N2 = N1*Vo2/Vd
L2 = Al*N2**2
N3 = N1*Vo3/Vd
L3 = Al*N3**2
print("")
print("\tN1 = ", N1, " vueltas")
print("\tL1 = ", L1*1E3, "mHy")
print("\tN2 = ", N2, " vueltas")
print("\tL2 = ", L2*1E6, "uHy")
print("\tN3 = ", N3, " vueltas")
print("\tL3 = ", L3*1E6, "uHy")
print("")

print("Opcion 2:")
IDC = 120E-3
coeffs = [IDC/(R*A_min), - Bsat, 0.5*Vd*D*Ts/(A_min)]
print("\tRaices:", np.roots(coeffs), ",\t Con IDC = 120mA")
N1 = 75

#Resto de parametros del transformador
Vo2 = 24
Vo3 = 16
L1 = Al*N1**2
N2 = N1*Vo2/Vd
L2 = Al*N2**2
N3 = N1*Vo3/Vd
L3 = Al*N3**2
print("")
print("\tN1 = ", N1, " vueltas")
print("\tL1 = ", L1*1E3, "mHy")
print("\tN2 = ", N2, " vueltas")
print("\tL2 = ", L2*1E6, "uHy")
print("\tN3 = ", N3, " vueltas")
print("\tL3 = ", L3*1E6, "uHy")
print("")

print("Opcion 3:")
IDC = 160E-3
coeffs = [IDC/(R*A_min), - Bsat, 0.5*Vd*D*Ts/(A_min)]
print("\tRaices:", np.roots(coeffs), ",\t Con IDC = 160mA")
N1 = 37.5

#Resto de parametros del transformador
Vo2 = 24
Vo3 = 16
L1 = Al*N1**2
N2 = N1*Vo2/Vd
L2 = Al*N2**2
N3 = N1*Vo3/Vd
L3 = Al*N3**2
print("")
print("\tN1 = ", N1, " vueltas")
print("\tL1 = ", L1*1E3, "mHy")
print("\tN2 = ", N2, " vueltas")
print("\tL2 = ", L2*1E6, "uHy")
print("\tN3 = ", N3, " vueltas")
print("\tL3 = ", L3*1E6, "uHy")
print("")

print("Opcion 4:")
IDC = 160E-3
coeffs = [IDC/(R*A_min), - Bsat, 0.5*Vd*D*Ts/(A_min)]
print("\tRaices:", np.roots(coeffs), ",\t Con IDC = 160mA")
N1 = 18.75

#Resto de parametros del transformador
Vo2 = 24
Vo3 = 16
L1 = Al*N1**2
N2 = N1*Vo2/Vd
L2 = Al*N2**2
N3 = N1*Vo3/Vd
L3 = Al*N3**2
print("")
print("\tN1 = ", N1, " vueltas")
print("\tL1 = ", L1*1E3, "mHy")
print("\tN2 = ", N2, " vueltas")
print("\tL2 = ", L2*1E6, "uHy")
print("\tN3 = ", N3, " vueltas")
print("\tL3 = ", L3*1E6, "uHy")
print("")


N_min =  18.42248908296943  vueltas. Redondeamos a  19.0
Raices: [2.43598098+9.1553079j 2.43598098-9.1553079j] ,	 Dan imaginarias, asi que la IDC es muy grande, probamos con algo mas bajo
Raices: [52.94633556 28.25303054] ,	 Con IDC = 180mA, tomamos N1=30

N1 =  30  vueltas
L1 =  4.68 mHy
N2 =  2.4  vueltas
L2 =  0.029952 mHy
N3 =  1.6  vueltas
L3 =  0.013312000000000003 mHy

Esta combinacion da valores de inductancias altas, pero peor es que da vueltas decimales, tenemos dos opciones:
	Opcion 1:
	Raices: [188.37896907  20.4194009 ] ,	 Con IDC = 70mA

	N1 =  150  vueltas
	L1 =  117.0 mHy
	N2 =  12.0  vueltas
	L2 =  748.8 uHy
	N3 =  8.0  vueltas
	L3 =  332.8 uHy

Opcion 2:
	Raices: [99.17367303 22.62537612] ,	 Con IDC = 120mA

	N1 =  75  vueltas
	L1 =  29.25 mHy
	N2 =  6.0  vueltas
	L2 =  187.2 uHy
	N3 =  4.0  vueltas
	L3 =  83.2 uHy

Opcion 3:
	Raices: [65.75677013 25.59251674] ,	 Con IDC = 160mA

	N1 =  37.5  vueltas
	L1 =  7.3125 mHy
	N2 =  3.0  vueltas
	L2 =  46.8 uHy
	N3 =  2.0  vu

***Snubber***
$$V_{DS} > V_{C_{SN}} > V_d + V_{o}\frac{N1}{N2}$$

$$C_{snubber} > \frac{L_{disp}*I_{L_{disp}}}{V_{sw_{max}^2 - (V_{d} + V_o \frac{N1}{N2})^2}}$$

$$R_{snubber} < \frac{T_{ON_{min}}}{3C_{snubber}} = \frac{D_{min}Ts}{3C_{snubber}}$$

In [50]:
#Estimacion de Ldisp REVISAR
Ldisp = 1E-3
ILdisp = 160E-3

Vsw_max = 900

Csn_min = Ldisp*ILdisp/(Vsw_max**2 - (Vd+Vo2*N1/N2)**2)
print("Csn_min = ", Csn_min*1E9, "nF")
Csn = 1E-9
print("Csn = ", Csn*1E9, "nF")
#Ahora para Rsn
Dmin = 0.3
Rsn_max = Dmin*Ts/(3*Csn)
print("Rsn_max = ", Rsn_max, "ohm")

Csn_min =  0.3555555555555556 nF
Csn =  1.0 nF
Rsn_max =  999.9999999999999 ohm
